# Data retrieval script

This notebook details the steps used to retrieve data from Lawline and save it to disk.

In [40]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import numpy as np
import pandas as pd

In [3]:
link_list = []

In [4]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

In [6]:
for _ in range(5):
    max_i = 10146

    for i in range(1, max_i):
        url = f"https://lawline.se/categories?page={i}"
        driver.get(url)
        time.sleep(3)
        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        links = soup.find_all('a',
                            attrs={'class': 'd-flex txt_grey flex-wrap'},
                            href=True)

        # Get links from page source
        for link in links:
            link_end = link.get("href")
            if link_end not in link_list:
                link_list.append(link_end)
        
        if i % 500 == 0:
            str_link_list = str(link_list)

            with open('link_list2.txt', 'w') as f:
                f.write(str_link_list)
            print(i)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500


WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=102.0.5005.115)
Stacktrace:
Backtrace:
	Ordinal0 [0x0065D953+2414931]
	Ordinal0 [0x005EF5E1+1963489]
	Ordinal0 [0x004DC6B8+837304]
	Ordinal0 [0x004D8E18+822808]
	Ordinal0 [0x004CDE5D+777821]
	Ordinal0 [0x004CEA3B+780859]
	Ordinal0 [0x004CE06A+778346]
	Ordinal0 [0x004CD646+775750]
	Ordinal0 [0x004CC565+771429]
	Ordinal0 [0x004CCA3D+772669]
	Ordinal0 [0x004DDEA4+843428]
	Ordinal0 [0x00534EBD+1199805]
	Ordinal0 [0x0052449C+1131676]
	Ordinal0 [0x00534812+1198098]
	Ordinal0 [0x005242B6+1131190]
	Ordinal0 [0x004FE860+976992]
	Ordinal0 [0x004FF756+980822]
	GetHandleVerifier [0x008CCC62+2510274]
	GetHandleVerifier [0x008BF760+2455744]
	GetHandleVerifier [0x006EEABA+551962]
	GetHandleVerifier [0x006ED916+547446]
	Ordinal0 [0x005F5F3B+1990459]
	Ordinal0 [0x005FA898+2009240]
	Ordinal0 [0x005FA985+2009477]
	Ordinal0 [0x00603AD1+2046673]
	BaseThreadInitThunk [0x76CF6739+25]
	RtlGetFullPathName_UE

In [8]:
str_link_list = str(link_list)

with open('link_list.txt', 'w') as f:
    f.write(str_link_list)

In [12]:
len(link_list)

101197

In [41]:
def get_fields(soup_in):
    # Date
    soup_date = soup_in.find(
        'span',
        attrs={
            'class': 'PublicQuestionItem_published_date__v9qn2'
            }
    ).text

    # Title
    soup_title = soup_in.find(
        'h1',
        attrs={
            'class': 'PublicQuestionItem_question_title__I4KwB'
            }
    ).text

    # Question
    soup_q = soup_in.find(
        'span',
        attrs={
            'class': 'PublicQuestionItem_question_text__dLAHK'
            }
    ).text

    # Answer
    soup_answer = soup_in.find(
        'span',
        attrs={
            'class': 'PublicQuestionDetails_answer_text__nhqcw'
            }
    ).text

    return soup_date, soup_title, soup_q, soup_answer

In [34]:
id_list = []
date_list = []
text_list = []
title_list = []
issues_list = []

In [48]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

In [52]:
id = 1

for link in link_list:

    url = f"https://lawline.se{link}"
    driver.get(url)
    time.sleep(2)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    try:
        soup_date, soup_title, soup_q, soup_answer = get_fields(soup)
    except:
        issues_list.append(link)
        continue

    if (soup_date is None or soup_title is None or
        soup_q is None or soup_answer is None):
        issues_list.append(link)
        continue
    else:
        id_list.extend([id, id])
        date_list.extend([soup_date, soup_date])
        text_list.extend([soup_q, soup_answer])
        title_list.extend([soup_title, soup_title])
        id += 1
    
    if id % 500 == 0:
        df = pd.DataFrame({'id': id_list, 'date': date_list,
                           'text': text_list, 'title': title_list})
        df.to_csv('lawline_data.csv', index=False)
        print(id)

In [57]:
df = pd.DataFrame({'id': id_list, 'date': date_list,
                   'text': text_list, 'title': title_list})

df_copy = df.copy()

df_copy = df_copy.drop_duplicates()

In [59]:
df_copy.to_csv('../dataset/lawline_data.csv', index=False)